# Assignment3: Recurrent Neural Networks
# Koç University COMP441-541

**Due on December 14th Sunday, 2025 (23:59:59)**

# Introduction

**Goals**

The primary goal with this assignment is to give you hands-on experience implementing a neural
network language model using recurrent neural networks. Understanding how these neural models work
and building one from scratch will help you understand not just language modeling, but also systems
for many other applications such as machine translation. Please use Python 3.5+ and PyTorch 1.0+ for
this project.

**Data**

The dataset for this paper is the text8 (http://mattmahoney.net/dc)
collection. This is a dataset taken from the first 100M
characters of Wikipedia. Only 27 character types are present (lowercase characters and spaces); special
characters are replaced by a single space and numbers are spelled out as individual digits (20 becomes
two zero). A larger version of this benchmark (90M training characters, 5M dev, 5M test) was used in
Mikolov et al. (2012).

# **Part1: RNNs for Classification (40 points)**

In this first part, you will do a simplified version of the language modeling task: binary classification of
fixed-length sequences to predict whether the given sequence is followed by a consonant or a vowel. You
will implement the entire training and evaluation loop for this model.

**Data**

*train-vowel-examples.txt* and *train-consonant-examples.txt* each contain 5000 strings of length
20, and *dev-vowel-examples.txt* and *dev-consonant-examples.txt* each contain 500. The task is to predict
whether the first letter following each string is a vowel or a consonant. The consonant file (for both train
and test) contains examples where the next letter (in the original text, not shown) was a consonant, and
analogously for the vowel file.

**Getting started**

Run: python lm classifier.py. This loads the data for this part, learns a FrequencyBasedClassifier on the data, and evaluates it. This classifier gets 71.4% accuracy, where random guessing
gets you 50%. lm classifier.py contains the driver code, and the top of models.py contains the skeletal
implementation for this classifier.

In [ ]:
class Indexer(object):
    """
    Bijection between objects and integers starting at 0. Useful for mapping
    labels, features, etc. into coordinates of a vector space.

    Attributes:
        objs_to_ints
        ints_to_objs
    """
    def __init__(self):
        self.objs_to_ints = {}
        self.ints_to_objs = {}

    def __repr__(self):
        return str([str(self.get_object(i)) for i in range(0, len(self))]) 

    def __str__(self):
        return self.__repr__()

    def __len__(self):
        return len(self.objs_to_ints)

    def get_object(self, index):
        """
        :param index: integer index to look up
        :return: Returns the object corresponding to the particular index or None if not found
        """
        if (index not in self.ints_to_objs):
            return None
        else:
            return self.ints_to_objs[index]

    def contains(self, object):
        """
        :param object: object to look up
        :return: Returns True if it is in the Indexer, False otherwise
        """
        return self.index_of(object) != -1

    def index_of(self, object):
        """
        :param object: object to look up
        :return: Returns -1 if the object isn't present, index otherwise
        """
        if (object not in self.objs_to_ints):
            return -1
        else:
            return self.objs_to_ints[object]

    def add_and_get_index(self, object, add=True):
        """
        Adds the object to the index if it isn't present, always returns a nonnegative index
        :param object: object to look up or add
        :param add: True by default, False if we shouldn't add the object. If False, equivalent to index_of.
        :return: The index of the object
        """
        if not add:
            return self.index_of(object)
        if (object not in self.objs_to_ints):
            new_idx = len(self.objs_to_ints)
            self.objs_to_ints[object] = new_idx
            self.ints_to_objs[new_idx] = object
        return self.objs_to_ints[object]


class Beam(object):
    """
    Beam data structure. Maintains a list of scored elements like a Counter, but only keeps the top n
    elements after every insertion operation. Insertion is O(n) (list is maintained in
    sorted order), access is O(1). Still fast enough for practical purposes for small beams.
    """
    def __init__(self, size):
        self.size = size  
        self.elts = []    
        self.scores = [] 

    def __repr__(self):
        return "Beam(" + repr(list(self.get_elts_and_scores())) + ")"

    def __str__(self):
        return self.__repr__()

    def __len__(self):
        return len(self.elts)

    def add(self, elt, score):
        """
        Adds the element to the beam with the given score if the beam has room or if the score
        is better than the score of the worst element currently on the beam

        :param elt: element to add
        :param score: score corresponding to the element
        """
        if len(self.elts) == self.size and score < self.scores[-1]:
            # Do nothing because this element is the worst
            return
        # If the list contains the item with a lower score, remove it
        i = 0
        while i < len(self.elts):
            if self.elts[i] == elt and score > self.scores[i]:
                del self.elts[i]
                del self.scores[i]
            i += 1
        # If the list is empty, just insert the item
        if len(self.elts) == 0:
            self.elts.insert(0, elt)
            self.scores.insert(0, score)
        # Find the insertion point with binary search
        else:
            lb = 0 
            ub = len(self.scores) - 1
            # We're searching for the index of the first element with score less than score
            while lb < ub:
                m = (lb + ub) // 2
                # Check > because the list is sorted in descending order
                if self.scores[m] > score:
                    # Put the lower bound ahead of m because all elements before this are greater
                    lb = m + 1
                else:
                    # m could still be the insertion point
                    ub = m
            # lb and ub should be equal and indicate the index of the first element with score less than score.
            # Might be necessary to insert at the end of the list.
            if self.scores[lb] > score:
                self.elts.insert(lb + 1, elt)
                self.scores.insert(lb + 1, score)
            else:
                self.elts.insert(lb, elt)
                self.scores.insert(lb, score)
            # Drop and item from the beam if necessary
            if len(self.scores) > self.size:
                self.elts.pop()
                self.scores.pop()

    def get_elts(self):
        return self.elts

    def get_elts_and_scores(self):
        return zip(self.elts, self.scores)

    def head(self):
        return self.elts[0]


##################
# Tests
def test_beam():
    print("TESTING BEAM")
    beam = Beam(3)
    beam.add("a", 5)
    beam.add("b", 7)
    beam.add("c", 6)
    beam.add("d", 4)
    print("Should contain b, c, a: %s" % beam)
    beam.add("e", 8)
    beam.add("f", 6.5)
    print("Should contain e, b, f: %s" % beam)
    beam.add("f", 9.5)
    print("Should contain f, e, b: %s" % beam)

    beam = Beam(5)
    beam.add("a", 5)
    beam.add("b", 7)
    beam.add("c", 6)
    beam.add("d", 4)
    print("Should contain b, c, a, d: %s" % beam)
    beam.add("e", 8)
    beam.add("f", 6.5)
    print("Should contain e, b, f, c, a: %s" % beam)

if __name__ == '__main__':
    test_beam()


TESTING BEAM
Should contain b, c, a: Beam([('b', 7), ('c', 6), ('a', 5)])
Should contain e, b, f: Beam([('e', 8), ('b', 7), ('f', 6.5)])
Should contain f, e, b: Beam([('f', 9.5), ('e', 8), ('b', 7)])
Should contain b, c, a, d: Beam([('b', 7), ('c', 6), ('a', 5), ('d', 4)])
Should contain e, b, f, c, a: Beam([('e', 8), ('b', 7), ('f', 6.5), ('c', 6), ('a', 5)])


In [275]:
import numpy as np
import collections

#####################
# MODELS FOR PART 1 #
#####################

class ConsonantVowelClassifier(object):
    def predict(self, context):
        """
        :param context:
        :return: 1 if vowel, 0 if consonant
        """
        raise Exception("Only implemented in subclasses")


class FrequencyBasedClassifier(ConsonantVowelClassifier):
    """
    Classifier based on the last letter before the space. If it has occurred with more consonants than vowels,
    classify as consonant, otherwise as vowel.
    """
    def __init__(self, consonant_counts, vowel_counts):
        self.consonant_counts = consonant_counts
        self.vowel_counts = vowel_counts

    def predict(self, context):
        # Look two back to find the letter before the space
        if self.consonant_counts[context[-1]] > self.vowel_counts[context[-1]]:
            return 0
        else:
            return 1

def train_frequency_based_classifier(cons_exs, vowel_exs):
    consonant_counts = collections.Counter()
    vowel_counts = collections.Counter()
    for ex in cons_exs:
        consonant_counts[ex[-1]] += 1
    for ex in vowel_exs:
        vowel_counts[ex[-1]] += 1
    return FrequencyBasedClassifier(consonant_counts, vowel_counts)

In [276]:
import argparse
import json
import time
from argparse import Namespace

############################################################
# DO NOT MODIFY THIS CELL IN YOUR FINAL SUBMISSION !!!!!!! #
############################################################


# def _parse_args():
#     """
#     Command-line arguments to the system. --model switches between the main modes you'll need to use. The other arguments
#     are provided for convenience.
#     :return: the parsed args bundle
#     """
#     parser = argparse.ArgumentParser(description='lm.py')
#     parser.add_argument('--model', type=str, default='FREQUENCY', help='model to run (FREQUENCY or RNN)')
#     parser.add_argument('--train_cons', type=str, default='data/train-consonant-examples.txt', help='path to train consonant examples')
#     parser.add_argument('--train_vowel', type=str, default='data/train-vowel-examples.txt', help='path to train vowel examples')
#     parser.add_argument('--dev_cons', type=str, default='data/dev-consonant-examples.txt', help='path to dev consonant examples')
#     parser.add_argument('--dev_vowel', type=str, default='data/dev-vowel-examples.txt', help='path to dev vowel examples')
#     parser.add_argument('--output_bundle_path', type=str, default='classifier-output.json', help='path to write the results json to (you should not need to modify)')
#     args = parser.parse_args()
#     return args


def read_examples(file):
    """
    :param file:
    :return: The text in the given file as a single string
    """
    all_lines = []
    for line in open(file):
        # Drop the last token (newline) but don't call strip() to keep whitespace
        all_lines.append(line[:-1])
        print(line[:-1])
    print("%i lines read in" % len(all_lines))
    return all_lines


def print_evaluation(dev_consonant_exs, dev_vowel_exs, model, output_bundle_path):
    """
    Runs the classifier on the given text
    :param text:
    :param lm:
    :return:
    """
    num_correct = 0
    for ex in dev_consonant_exs:
        if model.predict(ex) == 0:
            num_correct += 1
    for ex in dev_vowel_exs:
        if model.predict(ex) == 1:
            num_correct += 1
    num_total = len(dev_consonant_exs) + len(dev_vowel_exs)
    data = {'correct': num_correct, 'total': num_total, 'accuracy': float(num_correct)/num_total * 100.0}
    print("=====Results=====")
    print(json.dumps(data, indent=2))
    with open(output_bundle_path, 'w') as outfile:
        json.dump(data, outfile)


start_time = time.time()
#args = _parse_args()

args = Namespace(
    model='FREQUENCY',
    train_cons='data/train-consonant-examples.txt',
    train_vowel='data/train-vowel-examples.txt',
    dev_cons='data/dev-consonant-examples.txt',
    dev_vowel='data/dev-vowel-examples.txt',
    output_bundle_path='classifier-output.json'
)

print(args)

train_cons_exs = read_examples(args.train_cons)
train_vowel_exs = read_examples(args.train_vowel)
dev_cons_exs = read_examples(args.dev_cons)
dev_vowel_exs = read_examples(args.dev_vowel)

# Vocabs is lowercase letters a to z and space
vocab = [chr(ord('a') + i) for i in range(0, 26)] + [' ']
vocab_index = Indexer()
for char in vocab:
    vocab_index.add_and_get_index(char)
print(repr(vocab_index))

system_to_run = args.model
# Train our model
if system_to_run == "RNN":
    model = train_rnn_classifier(args, train_cons_exs, train_vowel_exs, dev_cons_exs, dev_vowel_exs, vocab_index)
elif system_to_run == "FREQUENCY":
    model = train_frequency_based_classifier(train_cons_exs, train_vowel_exs)
else:
    raise Exception("Pass in either UNIFORM or LSTM to run the appropriate system")

print_evaluation(dev_cons_exs, dev_vowel_exs, model, args.output_bundle_path)
# # print_evaluation(train_cons_exs[0:50], train_vowel_exs[0:50], model)

Namespace(dev_cons='data/dev-consonant-examples.txt', dev_vowel='data/dev-vowel-examples.txt', model='FREQUENCY', output_bundle_path='classifier-output.json', train_cons='data/train-consonant-examples.txt', train_vowel='data/train-vowel-examples.txt')
supplied to the cons
ther ballots nor bul
the spanish cnt as i
on on planting fores
t anarchists needed 
the most important t
 of the two in recen
d employment to matc
subjugation and domi
l liberty and proper
r and follow leo tol
ironment and equalit
velopment his brothe
state had opposing p
inst what they see a
r the months june th
ages between nine an
v h aminoglycoside a
form of government w
 global scale carrie
ce edward walker s s
ge of ideas includin
nions from this move
bbed pro cure those 
ertain of the bible 
ypto anarchism and c
d autism was first u
 environmental facto
and therefore less o
 points when he or s
ld is going to be di
e lowest of any natu
any articles by auti
tion a conflict whic
m in her books artic
ubgroups form

# **Q1 (40 points)**

Implement an RNN classifier to classify segments as being followed by consonants or
vowels. This will require defining a PyTorch module to do this classification, implementing training of
that module in train rnn classifier, and finally completing the definition of RNNClassifier appropriately
to use this module for classification. You can find the skeleton codes for RNNClassifier and train_rnn_classifier in the below cells.

**Important Note:** Your final model should get at least 75% accuracy. In your report, you should:
(1) Describe your model and implementation. (2) Report accuracy results and timing information. Even
if you are not able to get this part fully working, write up and document as much as you can so we can
give appropriate partial credit.

**Network Structure**

The inputs to your network will be sequences of character indices. You should
first embed these using a nn.Embedding layer and then feed the resulting tensor into an RNN. Two
effective types of RNNs to use are nn.GRU and nn.LSTM. Their weights should be randomly initialized
when you construct them, though you are also free to experiment with different initializers such as the
Glorot initializer (nn.init.xavier uniform). You should take the output of the RNN (the last hidden state)
and use it for binary classification with a softmax layer. You can add one or more feedforward layers
before the softmax layer if you want. You can make your own nn.Module that wraps the embedding
layer, RNN, and classification layer.

**Code Structure**

Once you have your own module implemented, the training and eval loop that wraps
them will look roughly the same as in previous assignments. First, you need a function
to go from the raw string to a PyTorch tensor of indices. Then loop through those examples, zero your
gradients, pick up an example, compute the loss, run backpropagation, and update parameters with your
optimizer. You should implement this training in train rnn classifier.

**Using RNNs**

LSTMs and GRUs can be a bit trickier to use than feedforward architectures. First,
these expect input tensors of dimension [sequence length, batch size, input size]. You can use the
batch first argument to switch whether the sequence length dimension or batch dimension occurs first.
If you’re not using batching, you’ll want to pad your sentence with a trivial 1 dimension for the batch.
unsqueeze allows you to add trivial dimensions of size 1, and squeeze lets you remove these. Second, an
LSTM takes as input a pair of tensors representing the state, h and c. Each is of size [num layers * num
directions, batch size, hidden size]. To start with, you probably want a 1-layer RNN just running in the
forward direction, so once again you should use unsqueeze to make a 3-tensor with first dimension length
of 1. GRUs are similar but only have one hidden state.

**Tensor Manipulation**

np.asarray can convert lists into numpy arrays easily. torch.from numpy can
convert numpy arrays into PyTorch tensors. torch.FloatTensor(list) can convert from lists directly to
PyTorch tensors. .float() and .int() can be used to cast tensors to different types.

**General Tips**

As always, make sure you can overfit a very small training set as an initial test. If not,
you probably have a bug. Then scale up to train on more data and check the development performance
of your model.cConsider using small values for hyperparameters so things train quickly. In particular,
with only 27 characters, you can get away with small embedding sizes for these, and small hidden sizes
for the RNN may work better than you think!

# Model:

I implemented an RNN classifier to predict the next character is consonant or vowel given a sequence. In my architecture I have:

1) Embedding layer: Maps the 27 characters to a 64-dimensional vector
2) RNN layer: Either LSTM or GRU
3) Fully connected layer: Classifies the last hidden state as consonant or vowel
4) Loss function: I used cross entropy loss

# Hyperparameters:

I tried various hyperparameters but I got the +75% accuracy using these hyperparameters:

- Embedding dimension: 64
- Hidden dimension: 128
- Batch size: 128
- Learning rate: 0.001
- Weight decay: 1e-4
- Number of epochs: 10

# Results:

| Model | Train Accuracy | Dev Accuracy | Training Time |
|-------|----------------|--------------|---------------|
| LSTM  | 83.65%         | 77.00%       | 8.38s         |
| GRU   | 83.24%         | 76.70%       | 7.13s         |

* Both models exceeded the required accuracy (75%). GRU trained faster than LSTM, it might be because GRU has fewer gates and because of that GRU has fewer parameters so it trains faster. LSTM achieved higher accuracy on dev set but the difference between two models are really close.

In [277]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [278]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, rnn_type):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn_type = rnn_type
        if rnn_type == "lstm":
            self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        elif rnn_type == "gru":
            self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        if self.rnn_type == "gru":
            _, hidden = self.rnn(embedded)
            last_hidden = hidden[-1]
        else:
            _, (hidden, cell_state) = self.rnn(embedded)
            last_hidden = hidden[-1]
        logits = self.fc(last_hidden)
        return logits


In [279]:
embedding_dim = 64
hidden_dim = 128
output_dim = 2
batch_size = 128
num_epochs = 10
learning_rate = 0.001
weight_decay = 1e-4

In [280]:
class RNNClassifier(ConsonantVowelClassifier):
    def __init__(self, model, vocab_index):
        self.model = model
        self.vocab_index = vocab_index
            
    def predict(self, context):
        self.model.eval()
    
        indices = [self.vocab_index.index_of(char) for char in context]
        tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(0)
        with torch.no_grad():
            logits = self.model.forward(tensor)
            probs = torch.softmax(logits, dim=1)
            prediction = torch.argmax(probs, dim=1).item()
        return prediction
            

In [281]:
def train_rnn_classifier(args, train_cons_exs, train_vowel_exs, dev_cons_exs, dev_vowel_exs, vocab_index, rnn_type):
    """
    :param args: command-line args, passed through here for your convenience
    :param train_cons_exs: list of strings followed by consonants
    :param train_vowel_exs: list of strings followed by vowels
    :param dev_cons_exs: list of strings followed by consonants
    :param dev_vowel_exs: list of strings followed by vowels
    :param vocab_index: an Indexer of the character vocabulary (27 characters)
    :return: an RNNClassifier instance trained on the given data
    """
    train_cons_indices = [[vocab_index.index_of(char) for char in word] for word in train_cons_exs]
    train_vowel_indices = [[vocab_index.index_of(char) for char in word] for word in train_vowel_exs]
    train_data = train_cons_indices + train_vowel_indices
    train_labels = [0] * len(train_cons_indices) + [1] * len(train_vowel_indices)

    dev_cons_indices = [[vocab_index.index_of(char) for char in word] for word in dev_cons_exs]
    dev_vowel_indices = [[vocab_index.index_of(char) for char in word] for word in dev_vowel_exs]
    dev_data = dev_cons_indices + dev_vowel_indices
    dev_labels = [0] * len(dev_cons_indices) + [1] * len(dev_vowel_indices)


    train_dataset = TensorDataset(torch.tensor(train_data, dtype=torch.long), torch.tensor(train_labels, dtype=torch.long))
    dev_dataset = TensorDataset(torch.tensor(dev_data, dtype=torch.long), torch.tensor(dev_labels, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)

    print(f"Creating model with model type: {rnn_type}")
    model = RNNModel(vocab_size=len(vocab_index), embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_dim=output_dim, rnn_type=rnn_type)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    start = time.time()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for data, labels in train_loader:
            optimizer.zero_grad()
            logits = model(data)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            prediction = torch.argmax(logits, dim=1)
            correct += (prediction == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
        training_time = time.time() - start
        print(f"\nTraining completed in {training_time:.2f} seconds")
    
        model.eval()
        dev_loss = 0.0
        dev_correct = 0
        dev_total = 0

        with torch.no_grad():
            for data, labels in dev_loader:
                logits = model(data)
                loss = criterion(logits, labels)
                dev_loss += loss.item()
                prediction = torch.argmax(logits, dim=1)
                dev_correct += (prediction == labels).sum().item()
                dev_total += labels.size(0)
            
        dev_loss = dev_loss / len(dev_loader)
        dev_acc = dev_correct / dev_total
        print(f"Dev Loss: {dev_loss:.4f}, Dev Accuracy: {dev_acc:.4f}")

    return RNNClassifier(model, vocab_index), dev_acc
            

In [284]:
rnn_types = ["lstm", "gru"]

for rnn_type in rnn_types:
    print("="*60)
    print(f"Training {rnn_type} model")
    print("="*60)
    model, dev_acc = train_rnn_classifier(args, train_cons_exs, train_vowel_exs, dev_cons_exs, dev_vowel_exs, vocab_index, rnn_type)


    filename = f"{rnn_type}_classifier_output.json"
    print(f"Evaluating {rnn_type}...")
    print_evaluation(dev_cons_exs, dev_vowel_exs, model, filename)



Training lstm model
Creating model with model type: lstm
Epoch 1/10, Train Loss: 0.5772, Train Accuracy: 0.7035

Training completed in 1.15 seconds
Dev Loss: 0.5212, Dev Accuracy: 0.7150
Epoch 2/10, Train Loss: 0.5073, Train Accuracy: 0.7421

Training completed in 2.12 seconds
Dev Loss: 0.4795, Dev Accuracy: 0.7550
Epoch 3/10, Train Loss: 0.4787, Train Accuracy: 0.7535

Training completed in 3.05 seconds
Dev Loss: 0.4610, Dev Accuracy: 0.7530
Epoch 4/10, Train Loss: 0.4598, Train Accuracy: 0.7659

Training completed in 3.95 seconds
Dev Loss: 0.4510, Dev Accuracy: 0.7680
Epoch 5/10, Train Loss: 0.4384, Train Accuracy: 0.7823

Training completed in 4.87 seconds
Dev Loss: 0.4370, Dev Accuracy: 0.7790
Epoch 6/10, Train Loss: 0.4190, Train Accuracy: 0.7904

Training completed in 5.79 seconds
Dev Loss: 0.4344, Dev Accuracy: 0.7770
Epoch 7/10, Train Loss: 0.4009, Train Accuracy: 0.8017

Training completed in 6.73 seconds
Dev Loss: 0.4376, Dev Accuracy: 0.7700
Epoch 8/10, Train Loss: 0.3840, T

# **Part 2: Implementing a Language Model (60 points)**

In this second part, you will implement an RNN language model. This should build heavily off of what
you did for Part 1, though new ingredients will be necessary, particularly during training.

**Data**

For this part, we use the first 100,000 characters of text8 as the training set. The development
set is 500 characters taken from elsewhere in the collection.

**Getting started**

Run the below cell. This loads the data, instantiates a UniformLanguageModel which assigns each character an equal 1
27 probability, and evaluates it on the development set. This model achieves
a total log probability of -1644, an average log probability (per token) of -3.296, and a perplexity of 27.
Note that exponentiating the average log probability gives you 1
27 in this case, which is the inverse of
perplexity.The RNNLanguageModel class you are given has two methods: get next char log probs and
get log prob sequence. The first takes a context and returns the log probability distribution over the
next characters given that context as a numpy vector of length equal to the vocabulary size. The second
takes a whole sequence of characters and a context and returns the log probability of that whole sequence
under the model. You can implement the second just using the first, but that’s computationally wasteful;
you can instead just run a single pass through the RNN and return the aggregated log probability of the
sequence.

In [285]:
class LanguageModel(object):

    def get_next_char_log_probs(self, context) -> np.ndarray:
        """
        Returns a log probability distribution over the next characters given a context.
        The log should be base e
        :param context: a single character to score
        :return: A numpy vector log P(y | context) where y ranges over the output vocabulary.
        """
        raise Exception("Only implemented in subclasses")


    def get_log_prob_sequence(self, next_chars, context) -> float:
        """
        Scores a bunch of characters following context. That is, returns
        log P(nc1, nc2, nc3, ... | context) = log P(nc1 | context) + log P(nc2 | context, nc1), ...
        The log should be base e
        :param next_chars:
        :param context:
        :return: The float probability
        """
        raise Exception("Only implemented in subclasses")


class UniformLanguageModel(LanguageModel):
    def __init__(self, voc_size):
        self.voc_size = voc_size

    def get_next_char_log_probs(self, context):
        return np.ones([self.voc_size]) * np.log(1.0/self.voc_size)

    def get_log_prob_sequence(self, next_chars, context):
        return np.log(1.0/self.voc_size) * len(next_chars)

In [286]:
import argparse
import json
import time

############################################################
# DO NOT MODIFY THIS FILE IN YOUR FINAL SUBMISSION !!!!!!! #
############################################################


# def _parse_args():
#     """
#     Command-line arguments to the system. --model switches between the main modes you'll need to use. The other arguments
#     are provided for convenience.
#     :return: the parsed args bundle
#     """
#     parser = argparse.ArgumentParser(description='lm.py')
#     parser.add_argument('--model', type=str, default='UNIFORM', help='model to run (UNIFORM or RNN)')
#     parser.add_argument('--train_path', type=str, default='data/text8-100k.txt', help='path to train set (you should not need to modify)')
#     parser.add_argument('--dev_path', type=str, default='data/text8-dev.txt', help='path to dev set (you should not need to modify)')
#     parser.add_argument('--output_bundle_path', type=str, default='output.json', help='path to write the results json to (you should not need to modify)')
#     args = parser.parse_args()
#     return args


def read_text(file):
    """
    :param file:
    :return: The text in the given file as a single string
    """
    all_text = ""
    for line in open(file):
        all_text += line
    print("%i chars read in" % len(all_text))
    return all_text


def run_sanity_check(lm, vocab_index):
    """
    Runs two sanity checks: (1) The language model must return valid probabilities for a few contexts. This checks that
    your model can take sequences of different lengths and contexts of different lengths without crashing.
    (2) Your reported next character distribution must agree with get_log_prob_sequence
    :param lm: the trained LM
    :return: True if the output is sane, false otherwise
    """
    contexts = [" ", " a person ", " some kind of person "]
    next_seqs = ["s", "sits", "sits there"]
    sane = True
    for context in contexts:
        for next_seq in next_seqs:
            log_prob = lm.get_log_prob_sequence(next_seq, context)
            if log_prob > 0.0:
                sane = False
                print("ERROR: sanity checks failed, LM log probability %f is invalid" % (log_prob))
            log_prob_from_single_probs = 0.0
            for i in range(0, len(next_seq)):
                # print(repr(next_seq[0:i]))
                # print(repr(next_seq[i]))
                next_char_log_probs = lm.get_next_char_log_probs(context + next_seq[0:i])
                # print(repr(next_char_log_probs))
                log_prob_from_single_probs += next_char_log_probs[vocab_index.index_of(next_seq[i])]
            if abs(log_prob_from_single_probs - log_prob) > 1e-3:
                sane = False
                print("ERROR: sanity checks failed, LM prob from sequence and single characters disagree: %f %f" % (log_prob, log_prob_from_single_probs))
    return sane


def normalization_test(lm, vocab_index):
    """
    Tests that LM normalizes, checks multiple contexts and sums over everything in the vocabulary to make sure it
    sums to one
    :param lm:
    :param voc:
    :return:
    """
    contexts = [" ", " a person "]
    normalizes = True
    for context in contexts:
        total_prob_single = np.sum(np.exp(lm.get_next_char_log_probs(context)))
        if total_prob_single < 0.99 or total_prob_single > 1.01:
            normalizes = False
            print("Probabilities sum to %f not 1.0 for context %s" % (total_prob_single, context))
        total_prob_seq = 0.0
        for char_idx in range(0, len(vocab_index)):
            total_prob_seq += np.exp(lm.get_log_prob_sequence(vocab_index.get_object(char_idx), context))
        if total_prob_seq < 0.99 or total_prob_seq > 1.01:
            normalizes = False
            print("Probabilities sum to %f not 1.0 for context %s" % (total_prob_seq, context))
    return normalizes


def print_evaluation(text, lm, vocab_index, output_bundle_path):
    """
    Runs both the sanity check and also runs the language model on the given text and prints three metrics: log
    probability of the text under this model (treating the text as one log sequence), average log probability (the
    previous value divided by sequence length), and perplexity (averaged "branching favor" of the model)
    :param text: the text to evaluate
    :param lm: model to evaluate
    :param output_bundle_path: the path to print the output bundle to, in addition to printing it
    """
    sane = run_sanity_check(lm, vocab_index)
    log_prob = lm.get_log_prob_sequence(text, " ")
    avg_log_prob = log_prob/len(text)
    perplexity = np.exp(-log_prob / len(text))
    # data = {'sane': sane, 'log_prob': log_prob, 'avg_log_prob': avg_log_prob, 'perplexity': perplexity}
    data = {'sane': sane, 'normalizes': normalization_test(lm, vocab_index), 'log_prob': log_prob, 'avg_log_prob': avg_log_prob, 'perplexity': perplexity}
    print("=====Results=====")
    print(json.dumps(data, indent=2))
    with open(output_bundle_path, 'w') as outfile:
        json.dump(data, outfile)



start_time = time.time()
args = Namespace(
    model='UNIFORM',
    train_path='data/text8-100k.txt',
    dev_path='data/text8-dev.txt',
    output_bundle_path='output.json'
)
print(args)

train_text = read_text(args.train_path)
dev_text = read_text(args.dev_path)

# Vocabs is lowercase letters a to z and space
vocab = [chr(ord('a') + i) for i in range(0, 26)] + [' ']
vocab_index = Indexer()
for char in vocab:
    vocab_index.add_and_get_index(char)
print(repr(vocab_index))

print("First 100 characters of train:")
print(train_text[0:100])
# Train our model
if args.model == "RNN":
    model = train_lm(args, train_text, dev_text, vocab_index)
elif args.model == "UNIFORM":
    model = UniformLanguageModel(len(vocab))
else:
    raise Exception("Pass in either UNIFORM or LSTM to run the appropriate system")

print_evaluation(dev_text, model, vocab_index, args.output_bundle_path)


Namespace(dev_path='data/text8-dev.txt', model='UNIFORM', output_bundle_path='output.json', train_path='data/text8-100k.txt')
100000 chars read in
500 chars read in
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']
First 100 characters of train:
 anarchism originated as a term of abuse first used against early working class radicals including t
=====Results=====
{
  "sane": true,
  "normalizes": true,
  "log_prob": -1647.9184330021646,
  "avg_log_prob": -3.295836866004329,
  "perplexity": 27.0
}


# **Q2 (60 points)**

Implement an RNN language model. This will require: defining a PyTorch module
to handle language model prediction, implementing training of that module in train_lm function, and finally
completing the definition of RNNLanguageModel appropriately to use this module for prediction. Skeleton codes for train_lm and RNNLanguageModel are provided to you in the cells below.
Your
network should take indexed characters as input, embed them, put them through an RNN, and make
predictions from the final layer outputs.

Your final model must pass the sanity and normalization checks, get a perplexity value less than
or equal to 7. In your report, you should: (1) Describe your model and implementation. (2) Report
accuracy results and any relevant time information. As with Part 1, even if you are not able to get this
part fully working, write up and document as much as you can so we can give appropriate partial credit.

**Chunking the Data**

Unlike classification, language modeling can be viewed as a task where the same
network is predicting words at many positions. Your network should process a chunk of characters at a
time, simultaneously predicting the next character at each index in the chunk. You’ll have to decide how
you want to chunk the data. Given a chunk, you can either initialize the RNN state with a zero vector,
“burn in” the RNN by running on a few characters before you begin predicting, or carry over the end
state of the RNN to the next state. These may only make minor differences, though.

**Start of Sequence**

In general, the beginning of any sequence is represented to the language model
by a special start-of-sequence token. This means that the inputs and outputs of a language model are
slightly different, since an LM will never output the start-of-sequence character but may need to read it
as input. For simplicity, we are going to overload space and use that as the start-of-sequence character.

**Evaluation**

In that part, your model is evaluated on perplexity and likelihood, which rely on the
probabilities that your model returns. Your model should be a correct implementation of a language
model. That is, it should be a probability distribution P(wi
|w1, ..., wi−1). You should be sure to check
that your model’s output is indeed a legal probability distribution over the next word.

**Batching**

Batching across multiple sequences can further increase the speed of training. While you do
not need to do this to complete the assignment, you may find the speedups helpful. As in Assignment 2,
you should be able to do this by increasing the dimension of your tensors by 1, a batch dimension which
should be the first dimension of each tensor. The rest of your code should be largely unchanged. Note
that you only need to apply batching during training, as the two inference methods you’ll implement
aren’t set up to pass you batched data anyway.


# Model:

I implemented a RNN language model to predict the next character by looking at the previous characters. My model architecture has:

1) Embedding layer: Maps each of the 27 characters to a 64-dimensional vector
2) RNN layer: Either LSTM or GRU 
3) Fully connected layer: Used to predict the next character
4) Loss function: NLLLoss

# Hyperparameters:

I trained both LSTM and GRU with different hyperparameters but I got the best results with:

- Embedding dimension: 64
- Hidden dimension: 128
- Sequence length: 50 characters per chunk
- Batch size: 64
- Learning rate: 0.001
- Weight decay: 1e-4
- Number of epochs: 10

### Sequence Length:

* I tried different numbers of chunks sizes (60, 55 , 52, 50), results are like this:

    * When sequence length was 60 both types of RNNs got the highest perplexity (LSTM's perplexity = 6.60, GRU's perplexity = 6.13), when I lowered the sequence length the perplexity values dropped. The best perplexity and other results are achieved using 50 for the sequence length.

### Chunking Strategy:
* I split both the training and dev text into fixed sized chunks which has 50 characters each. For the input chunk, it stores  characters from [0:49] (doesn't stre the character) and the target chunk stores characters from [1:50]. This allows model to predict the next character.

# Results:

| Model | Dev Accuracy | Total Training Time | Total Log Prob  | Avg Log Prob | Perplexity |
|-------|---------------------|---------------|--------------|--------------|--------|
| LSTM  |  45.33%             | ~5.2s | −914.36        | −1.8287      | **6.18**  |
| GRU   | 46.22%             | ~4.9s   | −858.28        | −1.7166      | **5.67**  |


* Both models met the required perplexity threshold(< 7).
* Both models passed the sanity and normalization checks
* GRU achieved better perplexity and higher average log probability compared to the LSTM, this means that GRU had a better predictive performance and also computationally more efficient than LSTMs
* Models accuracy is not that high but since we are evaluating the models for perplexity and likelihood, accuracies won't matter too much.





In [126]:
class RNNLmModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, rnn_type):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn_type = rnn_type
        if rnn_type == "lstm":
            self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        elif rnn_type == "gru":
            self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x, hidden=None):
        embedded = self.embedding(x)
        if self.rnn_type == "gru":
            output, hidden = rnn_out = self.rnn(embedded, hidden)
            
        else:
            output, (hidden, cell_state) = self.rnn(embedded, hidden)

        logits = self.fc(output)
        return self.log_softmax(logits)
            

In [287]:
class RNNLanguageModel(LanguageModel):
    def __init__(self, model, vocab_index):
        self.model = model
        self.vocab_index = vocab_index


    def get_next_char_log_probs(self, context):
        self.model.eval()
        
        indices = [self.vocab_index.index_of(char) for char in context]
        input = torch.tensor(indices, dtype=torch.long).unsqueeze(0)
        with torch.no_grad():
            log_probs = self.model.forward(input)
        
        last_log_prob = log_probs[0, -1, :]
        return last_log_prob.cpu().numpy()

    def get_log_prob_sequence(self, next_chars, context):
        
        self.model.eval()
        full_sequence = context + next_chars
        indices = [self.vocab_index.index_of(char) for char in full_sequence]
        input = indices[:-1]
        target = indices[len(context):]
        input_tensor = torch.tensor(input, dtype=torch.long).unsqueeze(0)
        with torch.no_grad():
            log_probs = self.model.forward(input_tensor)

        total = 0.0
        for i, target_index in enumerate(target):
            log_prob = log_probs[0, len(context) - 1 + i, target_index]
            total += log_prob.item()
        return total


In [288]:
embedding_dim = 64
hidden_dim = 128
batch_size = 64
num_epochs = 10
learning_rate = 0.001
seq_len = 50
weight_decay = 1e-4
vocab_size = len(vocab_index)

In [289]:
def create_chunks(text, seq_len):
    input_chunks = []
    target_chunks = []
    for i in range(0, len(text) - seq_len, seq_len):
        chunk = text[i : i + seq_len + 1]
        if len(chunk) == seq_len + 1:
            input_chunks.append(chunk[:-1])
            target_chunks.append(chunk[1:])

    input_tensor = torch.tensor(input_chunks, dtype=torch.long)
    target_tensor = torch.tensor(target_chunks, dtype=torch.long)
    return TensorDataset(input_tensor, target_tensor)


In [ ]:
def train_lm(args, train_text, dev_text, vocab_index):
    """
    :param args: command-line args, passed through here for your convenience
    :param train_text: train text as a sequence of characters
    :param dev_text: dev texts as a sequence of characters
    :param vocab_index: an Indexer of the character vocabulary (27 characters)
    :return: an RNNLanguageModel instance trained on the given data
    """

    print("Preparing Training Data...")
    train_indices = [vocab_index.index_of(c) for c in train_text]
    train_dataset = create_chunks(train_indices, seq_len)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    print("Preparing Dev Data...")
    dev_indices = [vocab_index.index_of(c) for c in dev_text]
    dev_dataset = create_chunks(dev_indices, seq_len)
    dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False) 

    model = RNNLmModel(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_dim=hidden_dim, rnn_type=args.model)
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    print(f"Training {args.model.upper()} Model on {len(train_dataset)} chunks...")
    print("-"*60)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0
        start_time = time.time()

        for input, target in train_loader:
            optimizer.zero_grad()
            output = model(input)
            loss = criterion(output.view(-1, vocab_size), target.view(-1))
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            predictions = output.argmax(dim=2) 
            train_correct += (predictions == target).sum().item()
            train_total += target.numel()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        epoch_time = time.time() - start_time

        model.eval()
        dev_loss = 0.0
        dev_correct = 0
        dev_total = 0
        
        with torch.no_grad():
            for input, target in dev_loader:
                output = model(input)
                loss = criterion(output.view(-1, vocab_size), target.view(-1))
                dev_loss += loss.item()
                predictions = output.argmax(dim=2)
                dev_correct += (predictions == target).sum().item()
                dev_total += target.numel()

        average_dev_loss = dev_loss / len(dev_loader)
        dev_accuracy = 100 * dev_correct / dev_total
        dev_perplexity = np.exp(average_dev_loss)
        

        print(f"Epoch {epoch+1}/{num_epochs} | Time: {epoch_time:.1f}s | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_accuracy:.2f}% | "
              f"Dev Loss: {average_dev_loss:.4f} | Dev Acc: {dev_accuracy:.2f}% | "
              f"Dev PPL: {dev_perplexity:.2f}")

    print("-"*60)
    print(f"Training Complete!")
    print(f"Final Train Accuracy: {train_accuracy:.2f}%")
    print(f"Final Dev Accuracy: {dev_accuracy:.2f}%")
    print(f"Final Dev Perplexity: {dev_perplexity:.2f}")
    
    return RNNLanguageModel(model, vocab_index)

In [150]:
rnn_types = ["lstm", "gru"]

for rnn_type in rnn_types:
    print("="*60)
    print(f"Training {rnn_type} model")
    print("="*60)
    args.model = rnn_type 
    lm = train_lm(args, train_text, dev_text, vocab_index)
    
    output_file = f"{rnn_type}_lm_output.json"
    print(f"\nRunning Final Evaluation for {rnn_type}...")
    print_evaluation(dev_text, lm, vocab_index, output_file)



Training lstm model
Preparing Training Data...
Preparing Dev Data...
Training LSTM Model on 1999 chunks...
------------------------------------------------------------
Epoch 1/10 | Time: 0.6s | Train Loss: 2.9203 | Train Acc: 18.05% | Dev Loss: 2.5847 | Dev Acc: 25.78% | Dev PPL: 13.26
Epoch 2/10 | Time: 0.6s | Train Loss: 2.5230 | Train Acc: 27.10% | Dev Loss: 2.3133 | Dev Acc: 30.44% | Dev PPL: 10.11
Epoch 3/10 | Time: 0.7s | Train Loss: 2.3309 | Train Acc: 30.77% | Dev Loss: 2.1711 | Dev Acc: 33.33% | Dev PPL: 8.77
Epoch 4/10 | Time: 0.6s | Train Loss: 2.2177 | Train Acc: 33.89% | Dev Loss: 2.0962 | Dev Acc: 36.00% | Dev PPL: 8.13
Epoch 5/10 | Time: 0.6s | Train Loss: 2.1332 | Train Acc: 36.67% | Dev Loss: 2.0261 | Dev Acc: 39.11% | Dev PPL: 7.58
Epoch 6/10 | Time: 0.6s | Train Loss: 2.0631 | Train Acc: 38.67% | Dev Loss: 1.9695 | Dev Acc: 39.78% | Dev PPL: 7.17
Epoch 7/10 | Time: 0.6s | Train Loss: 2.0058 | Train Acc: 40.25% | Dev Loss: 1.9291 | Dev Acc: 41.78% | Dev PPL: 6.88
Epoc

## Part 3 (Bonus 20 pts): Needle in a Haystack Problem

Now, we introduce a recent problem known as the Needle in a Haystack Problem. Here, we will take a dataset, and at random positions in each input, we will insert text of the form `Passcode: N` where N is a random digit from `0` to `9`. The goal of the problem is to give the entire input to the model, and have the model output what the passcode digit is.

For example, if the original input is `First, you know Caius Marcius is chief enemy to the people.`, we might process this to be `First, you know Caius Marcius Passcode: 5 is chief enemy to the people.`, and want the model to classify this text as `5`. Wherever this passcode is added, make sure to put spaces before and after.

We will use the shakespeare dataset introduced in Tutorial 6.

### Step 1: Prepare the Dataset and Processor
You will need to load the dataset. To do so, you can adapt the code provided in Tutorial 6. However, you will need to write a function called `add_passcode(text)` which adds a random passcode to the text. For this function, make sure to return both the text with the passcode added, as well as the passcode itself (You will need this as a label for training). After this, you will also need to be able to convert the text into ids.

### Step 2: Setup and Train the Model
To train a model on this task, you can adapt the classification model from Part 1. It will need to predict 10 classes corresponding to the 10 digits. Experiment with a vanilla RNN, LSTM, and GRU, as well as different settings, and determine what the best performance you can get on this task is. Report all of these results for different architectures and training setups.

** This is a challenging task, do not expect to be able to get high results with a small dataset and small model **

# Models and Results:

* My initial models were:

1) Vanilla Rnn
2) LSTM RNN
3) GRU RNN

* Initially I tried the train all the models with the whole dataset but because the dataset was too big for my computer, training were taking too long and I decided to use 5000 examples for training set and 1000 examples for the dev set. I initially trained the models with these hyperparameters:

- embedding_dim = 32
- hidden_dim = 64
- output_dim = 10
- batch_size = 128
- num_epochs = 15
- learning_rate = 0.001
- optimizer: Adam optimizer

* Initally I did not used weight decay because when I added weight decay the accuracies dropped down significantly but after removing weight decay GRU got 100% accuracy on the dev set so I decided to add the weight decay again. The best results I got with hyperparameters above + weight decay = 1e-4 is like this:

| Model | Dev Accuracy |
|-------|--------------|
| RNN   | 10.60%       |
| LSTM  | 53.40%       |
| GRU   | 53.20%       |


* LSTM and GRU significantly outperformed vanilla RNN, achieved nearly 5 times better accuracy then vanilla RNN.
* Vanilla RNN might not have a good performence because of the vanishing gradient problem and also the low accuracy might be because, I padded the sentences in the batches so that they have the same length. The more I padded the more time vanilla RNN spends time to understand what does the padding mean and lose important information.

## Full training/dev set:

* After training the models with 5k training examples and 1k examples for the dev set. I decided to train the models with 21147 training set examples and 2350 dev set examples (I split the training/dev set according to tutorial 6). I trained the same models (vanilla RNN, LSTM, GRU), with the same hyperparameters above (with weight decay). The results were like this:


| Model | Dev Accuracy | Improvement |
|-------|--------------|-------------|
| RNN   | 10.17%       | -0.43%   |
| LSTM  | 69.36%       | +15.96%     |
| GRU   | **99.91%**   | +46.71%     |

* Accuracy of the vanilla RNN did not change when the data size incerased. It still randomly classified the numbers (10% for 10 classes)
* LSTM's accuracy improved (from 53.40% to 89.74%). This indicates that LSTM benefits from using more data.
* GRU achieved near perfect accuracy. This might be happened because GRU might learned to detect "Passcode: " and focused only on the number.
* The near perfect accuracy suggests that GRU is the best model for this task.


### Additional on GRU's performance

* I wanted to invastigate and play with GRU's accuracy, added various hyperparameters like dropout with various hyperparameters, increased weight decay. 

* One of two things happened. It was either GRU's achieved high accuracies (85%-99%) and LSTMS accuracy dropped significantly (dropped nearly to 10 %) or both GRU and LSTM achieved worse accuracies (both achieved nearly 10%). This strongly indicates that GRU is the most suitable model for the task.

## Conclusion:

**Data Size**: I understood how important is the data size one more time. Since both LSTM and GRU improved their accuracies with more data.

**Architecture**: LSTM and GRU are better for tasks that require capturing long range dependencies. Gating mechanisms helped them. Vanilla RNNs suffer from vanishing gradient problem and doesn't learn anything, randomly classifies the given text.


In [292]:
def load_shakespeare_data(file_path='shakespeare.txt'):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

In [293]:
def create_vocab_index(text):
    vocab = sorted((list(set(text))))

    passcode_chars = ['P', 'a', 's', 's', 'c', 'o', 'd', 'e', ':', ' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    for char in passcode_chars:
        if char not in vocab:
            vocab.append(char)
    
    vocab = sorted(vocab)
    
    vocab_index = Indexer()
    for char in vocab:
        vocab_index.add_and_get_index(char)
    return vocab_index


In [294]:
import random

def add_passcode(text):
    passcode = "Passcode: " + str(random.randint(0, 9))
    start_index = random.randint(0, len(text) - 1)
    space = [i for i, char in enumerate(text) if char == " "]
    if not space:
        insert_position = len(text)
    else:
        insert_position = random.choice(space)

    text = text[:insert_position] + " " + passcode + " " + text[insert_position:]
    return text, int(passcode[-1])

    
def encode_text(text, vocab_index):
    return [vocab_index.index_of(char) for char in text]

In [295]:
text = "First, you know Caius Marcius is chief enemy to the people."
modified, digit = add_passcode(text)

print(f"Original: {text}")
print(f"Modified: {modified}")


Original: First, you know Caius Marcius is chief enemy to the people.
Modified: First, you know Caius Marcius is chief enemy to the Passcode: 1  people.


In [296]:
shakespeare_text = load_shakespeare_data('data/shakespeare.txt')
print(f"Total characters: {len(shakespeare_text)}")

vocab_index = create_vocab_index(shakespeare_text)
print(f"Vocabulary size: {len(vocab_index)}")

lines = [line.strip() for line in shakespeare_text.split('\n') if len(line.strip()) > 20]
print(f"Total lines: {len(lines)}")

random.shuffle(lines)
split_index = int(len(lines) * 0.9)
train_lines = lines[:split_index]
dev_lines = lines[split_index:]

#train_lines = lines[:5000]
#dev_lines = lines[5000:6000]

print(f"Train examples: {len(train_lines)}")
print(f"Dev examples: {len(dev_lines)}")

Total characters: 1115394
Vocabulary size: 74
Total lines: 23497
Train examples: 21147
Dev examples: 2350


In [297]:
def prepare_shakespeare_data(text, vocab_index, max_length):
    input = []
    label = []
    for line in text:
        if len(line) > max_length:
            line = line[:max_length]    
        modified_text, digit = add_passcode(line)
        encoded_text = encode_text(modified_text, vocab_index)

        fixed_length = max_length + 10
        if len(encoded_text) < fixed_length:
            encoded_text = encoded_text + [0] * (fixed_length - len(encoded_text))
        else:
            encoded_text = encoded_text[:fixed_length]
            
        input.append(encoded_text)
        label.append(digit)
    return input, label

train_data, train_labels = prepare_shakespeare_data(train_lines, vocab_index, 50)
dev_data, dev_labels = prepare_shakespeare_data(dev_lines, vocab_index, 50)

train_dataset = TensorDataset(torch.tensor(train_data), torch.tensor(train_labels))
dev_dataset = TensorDataset(torch.tensor(dev_data), torch.tensor(dev_labels))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=64, shuffle=False)

In [298]:
embedding_dim = 64
hidden_dim = 128
output_dim = 10
batch_size = 128
num_epochs = 5
learning_rate = 0.001
weight_decay = 1e-4

In [299]:
class PasscodeRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, rnn_type):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn_type = rnn_type    
        if rnn_type == "lstm":
            self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        elif rnn_type == "gru":
            self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        elif rnn_type == "rnn":
            self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x, hidden=None):
        embedded = self.embedding(x)
        if self.rnn_type == "lstm":
            output, (hidden, cell_state) = self.rnn(embedded)
            last_hidden = hidden[-1]
        elif self.rnn_type == "gru":
            output, hidden  = self.rnn(embedded)
            last_hidden = hidden[-1]
        elif self.rnn_type == "rnn":
            output, hidden  = self.rnn(embedded)
            last_hidden = hidden[-1]

        logits = self.fc(last_hidden)
        return logits
        

In [300]:
def train_passcode_rnn(train_loader, dev_loader, vocab_index, rnn_type):
    model = PasscodeRNN(vocab_size=len(vocab_index), embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_dim=output_dim, rnn_type=rnn_type)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    print(f"Training {rnn_type.upper()} Model on {len(train_dataset)} chunks...")
    print("-"*60)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for data, labels in train_loader:
            optimizer.zero_grad()
            logits = model(data)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            prediction = torch.argmax(logits, dim=1)
            train_correct += (prediction == labels).sum().item()
            train_total += labels.size(0)


        model.eval()
        dev_loss = 0.0
        dev_correct = 0
        dev_total = 0

        with torch.no_grad():
            for data, labels in dev_loader:
                logits = model(data)
                loss = criterion(logits, labels)
                dev_loss += loss.item()
                prediction = torch.argmax(logits, dim=1)
                dev_correct += (prediction == labels).sum().item()
                dev_total += labels.size(0)

        train_acc = train_correct / train_total      
        dev_acc = dev_correct / dev_total 
                
        print(f"Epoch {epoch+1}/{num_epochs} | Train Acc: {train_acc * 100:.2f}% | Dev Acc: {dev_acc * 100:.2f}%")
    
    return model, dev_correct/dev_total
                

In [302]:
results = {}

for rnn_type in ['rnn', 'lstm', 'gru']:
    print(f"\n{'='*50}")
    print(f"Training {rnn_type.upper()}")
    print('='*50)
    
    model, dev_acc = train_passcode_rnn(
        train_loader, dev_loader, 
        vocab_index,
        rnn_type=rnn_type
    
    )
    results[rnn_type] = dev_acc

print("\n=== Final Results ===")
for rnn_type, acc in results.items():
    print(f"{rnn_type.upper()}: {acc*100:.2f}%")


Training RNN
Training RNN Model on 21147 chunks...
------------------------------------------------------------
Epoch 1/5 | Train Acc: 10.14% | Dev Acc: 10.47%
Epoch 2/5 | Train Acc: 10.78% | Dev Acc: 10.34%
Epoch 3/5 | Train Acc: 11.17% | Dev Acc: 9.53%
Epoch 4/5 | Train Acc: 11.36% | Dev Acc: 9.87%
Epoch 5/5 | Train Acc: 11.44% | Dev Acc: 10.60%

Training LSTM
Training LSTM Model on 21147 chunks...
------------------------------------------------------------
Epoch 1/5 | Train Acc: 9.99% | Dev Acc: 9.96%
Epoch 2/5 | Train Acc: 12.12% | Dev Acc: 14.17%
Epoch 3/5 | Train Acc: 34.78% | Dev Acc: 38.17%
Epoch 4/5 | Train Acc: 52.58% | Dev Acc: 59.96%
Epoch 5/5 | Train Acc: 64.00% | Dev Acc: 69.36%

Training GRU
Training GRU Model on 21147 chunks...
------------------------------------------------------------
Epoch 1/5 | Train Acc: 10.24% | Dev Acc: 10.89%
Epoch 2/5 | Train Acc: 48.27% | Dev Acc: 99.91%
Epoch 3/5 | Train Acc: 99.96% | Dev Acc: 99.91%
Epoch 4/5 | Train Acc: 99.96% | Dev Acc